# 类与对象

## 改变对象的字符串显示
问题：想改变对象实例的打印内容，增加可读性
解决：重新定义 __ str __ () 和 __ repr __ () 方法

- 使用 repr() 函数返回 __ repr __ () 方法定义的字符串
- 使用 str() 或 print() 函数会返回 __ str __ () 方法定义的字符串

In [5]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'MyPair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

p = Pair(3, 4)
print(repr(p))
print(p)

MyPair(3, 4)
(3, 4)


## 自定义字符串的格式化
问题：想通过 format() 函数和字符串方法使得一个对象能支持自定义的格式化
解决：在类上面定义 format () 方法

In [7]:
_formats = { 'ymd' : '{d.year}-{d.month}-{d.day}', 
             'mdy' : '{d.month}/{d.day}/{d.year}', 
             'dmy' : '{d.day}/{d.month}/{d.year}' }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [9]:
d = Date(2012, 12, 21)
print(format(d))
print('The date is {:ymd}'.format(d))
print('The date is {:mdy}'.format(d))

2012-12-21
The date is 2012-12-21
The date is 12/21/2012


In [10]:
from datetime import date
d = date(2012, 12, 21)
print(format(d))
print(format(d,'%A, %B %d, %Y'))

2012-12-21
Friday, December 21, 2012


## 创建大量对象时节省内存方法
问题：程序要创建大量 (可能上百万) 的对象，导致占用很大的内存，怎样压缩空间？
解答：给类添加 __ slots __ 属性来极大的减少实例所占的内存
- 优点：这样实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典
- 不能再给实例添加新的属性了，只能使用在 slots 中定义的那些属性名
- 应用场景：需要创建某个类的几百万个实例对象

In [ ]:
class Date:
    __slots__ = ['year', 'month', 'day']
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        

## 私有方法
问题：如何私有化方法
解决：约定是任何以下划线"_"开头的名字都是私有方法
- 以单下划线开头的方法可以被子类访问，如果被覆写了，则访问子类的方法
- 以双下划线开头的方法不能被子类访问
- tip: 如果变量名/属性名与关键字冲突，可以在后面加下划线，如：lambda_

In [1]:
class A:
    def __init__(self):
        self._internal = 0 # 私有属性
        self.public = 1 # 公共属性
        
    def public_method(self):
        '''
        A public method
        '''
        pass

    def _private_method(self):
        '''
        A private method
        '''
        print("A's _private_method")

class B(A):
    def __init__(self):
        super().__init__()
        self._private_method() # 如果子类不覆写，则直接调用父类的方法
        self._internal = 2

    # def _private_method(self):
    #     print("B's _private_method")

b = B()

A's _private_method


In [2]:
class C:
    def __init__(self):
        self.__private = 0
    def __private_method(self):
        print("C's __private_method")
        print("C's __private is "+str(self.__private))
    def public_method(self):
        pass
        self.__private_method()


class D(C):
    def __init__(self):
        super().__init__()
        self._C__private = 1
        self.__private_method() # 没有被子类覆写，也不能调用父类的方法

    # Does not override C.__private_method()
    # def __private_method(self):
    #     print("D's __private_method")

d = D()

AttributeError: 'D' object has no attribute '_D__private_method'

## 创建可管理的属性
问题：想给某个实例 attribute 增加除访问与修改之外的其他处理逻辑，比如类型检查或合法性验证
解决：将属性定义为一个 property

In [6]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    # Getter，使得 first_name 成为一个属性
    @property
    def first_name(self):
        print("调用了Getter方法")
        return self._first_name

    # Setter，给 first_name 属性添加了 setter
    @first_name.setter
    def first_name(self, value):
        print("调用了Setter方法")
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value # 底层数据实际保存的地方，另一个属性

    # Deleter，给 first_name 属性添加了 deleter
    @first_name.deleter
    def first_name(self):
        print("调用了Deleter方法")
        raise AttributeError("Can't delete attribute")

p = Person("Jason")
print(p.first_name)

调用了Setter方法
调用了Getter方法
Jason


注意：
- Java 程序员总认为所有访问都应该通过 getter 和 setter
- 例如下面这种写法，这毫无意义，代码臃肿，程序变慢
- 如果以后想给普通属性访问添加额外逻辑时，可以再添加@property，访问这个属性的代码保持不变

In [ ]:
# 不好的案例
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = value

## 调用父类的方法
问题：在子类中调用父类的某个已经被覆写的方法
解决：使用super()函数

场景一：调用父类的 __ init __ 方法，确保父类被正确的初始化了

In [ ]:
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

场景二：出现在覆盖 Python 特殊方法的代码中

In [ ]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        return getattr(self._obj, name)

    # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value) # Call original __setattr__
        else:
            setattr(self._obj, name, value)

注意：在调用父类的方法时，避免直接通过父类类名调用

In [ ]:
# 错误的案例
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

如果没有父类，则会报错：

In [2]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()

a = A()
a.spam()

A.spam


AttributeError: 'super' object has no attribute 'spam'

但是如果是多继承，会很神奇地不会报错

In [3]:
class B:
    def spam(self):
        print('B.spam')

class C(A,B):
    pass

c = C()
c.spam()

A.spam
B.spam


## 属性的懒加载
问题：如何让一个属性在第一次使用时才计算，后面缓存起来
解决：使用一个描述器类来实现

In [9]:
class lazyproperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
        setattr(instance, self.func.__name__, value)
        return value

import math

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

c = Circle(4.0)
print(vars(c))

print(c.area)
# 查看对象c，发现c.area已经有值了
print(vars(c))
# 第二次调用 c.area，只会输出一次"Computing area"，说明并没有真正地
print(c.area)

# 删除属性
del c.area
print(vars(c))



{'radius': 4.0}
Computing area
50.26548245743669
{'radius': 4.0, 'area': 50.26548245743669}
50.26548245743669
{'radius': 4.0}


- 当一个描述器被放入一个类的定义时，每次访问属性时它的 __ get __()、 __ set __() 和 __ delete __() 方法就会被触发
- 如果一个描述器仅仅只定义了一个 __ get __() 方法的话，它比通常的具有更弱的绑定：只有当被访问属性不在实例底层的字典中时 __ get __() 方法才会被触发

## 简化数据结构的初始化
问题：写了很多仅仅用作数据结构的类，不想写太多烦人的 __ init __() 函数
解决：在一个基类中写一个公用的 __ init __() 函数

In [10]:
import math
class Structure1: # 基类
    _fields = []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)

# 其他类继承基类
class Stock(Structure1):
    _fields = ['name', 'shares', 'price']

class Point(Structure1):
    _fields = ['x', 'y']

class Circle(Structure1):
    _fields = ['radius']
    def area(self):
        return math.pi * self.radius ** 2

s = Stock('ACME', 50, 91.1)
p = Point(2, 3)
c = Circle(4.5)
s2 = Stock('ACME', 50)

TypeError: Expected 3 arguments